In [ ]:
import yaml
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Optional
import json
import warnings
warnings.filterwarnings('ignore')

from evidently import Report
from evidently.presets import DataDriftPreset, ClassificationPreset
from sklearn.metrics import roc_auc_score
import scorecardpy as sc
import os, sys


# reload

In [ ]:
SCR_PATH = 'D:/WORK_F88/Tracy/Projects/001_evidently_monitor_observation'
sys.path.append(SCR_PATH)

from src.monitoring.utils import (load_config)
from src.monitoring.metrics import (
    AUCMetric,
    GiniMetric,
    DefaultRateMetric,
    KSMetric,
)
import importlib
import src.monitoring.pipeline
importlib.reload(src.monitoring.pipeline)
from src.monitoring.pipeline import GenericModelMonitor

In [ ]:


monitor_churn = GenericModelMonitor('D:/WORK_F88/Tracy/Projects/001_evidently_monitor_observation/config/monitoring_config.yaml')

reference_path_predict='D:/WORK_F88/Tracy/Projects/001_evidently_monitor_observation/data/training data/result_20260105_164657_20250625.parquet'
current_path_predict='D:/WORK_F88/Tracy/Projects/001_evidently_monitor_observation/data/result_20260105_164657_20251125.parquet'
period='2025-06'
reference_path=r'D:\WORK_F88\Tracy\Projects\001_evidently_monitor_observation\data\ref_all.parquet'
current_path=r'D:\WORK_F88\Tracy\Projects\001_evidently_monitor_observation\data\cur_all.parquet'

In [ ]:
reference_df, current_df = monitor_churn.load_data(
        reference_path=reference_path, 
        current_path=current_path
        )
reference_df_pred, current_df_pred = monitor_churn.load_data(
        reference_path=reference_path_predict, 
        current_path=current_path_predict
        )


In [ ]:
from src.monitoring.utils import (load_config)
from evidently import DataDefinition
from evidently import Dataset
from evidently import Regression, Report
from evidently.metrics import ValueDrift


In [ ]:
ref_df = pd.read_parquet(reference_path)
curr_df = pd.read_parquet(current_path)
curr_df = curr_df.sample(frac=1, random_state=42).reset_index()
ref_df = ref_df.sample(frac=1, random_state=20).reset_index()
print(f"  Reference: {ref_df.shape}")
print(f"  Current: {curr_df.shape}")

definition = DataDefinition(
    # text_columns=categorical_columns,
    numerical_columns=monitor_churn.numerical_columns + [monitor_churn.predict_column] + [monitor_churn.target_column],
    categorical_columns=monitor_churn.categorical_columns,
    # datetime_columns=monitor_churn.datetime_columns,
    id_column=monitor_churn.id_column,
#     timestamp=monitor_churn.timestamp_column,
    regression=[Regression(target=monitor_churn.target_column, 
                        prediction=monitor_churn.predict_column
                        )
                ]
)

ref_df = Dataset.from_pandas(
        ref_df,
        data_definition=definition
        )
curr_df = Dataset.from_pandas(
        curr_df,
        data_definition=definition
        )


In [ ]:

report = Report(
    [
        ValueDrift(column=monitor_churn.predict_column, method="psi"),
        # DataDriftPreset(
        #     num_method="psi",
        #     cat_method="psi",
        #     include_tests=True
        # )
    ],
)
my_eval = report.run(current_data=curr_df, reference_data=ref_df)
my_eval

In [ ]:
ref_df = pd.read_parquet(reference_path)
curr_df = pd.read_parquet(current_path)
curr_df = curr_df.sort_values("CUSTOMER_CODE").reset_index()
ref_df = ref_df.sort_values("CUSTOMER_CODE").reset_index()
print(f"  Reference: {ref_df.shape}")
print(f"  Current: {curr_df.shape}")

definition = DataDefinition(
    # text_columns=categorical_columns,
    numerical_columns=monitor_churn.numerical_columns + [monitor_churn.predict_column] + [monitor_churn.target_column],
    categorical_columns=monitor_churn.categorical_columns,
    # datetime_columns=monitor_churn.datetime_columns,
    id_column=monitor_churn.id_column,
#     timestamp=monitor_churn.timestamp_column,
    regression=[Regression(target=monitor_churn.target_column, 
                        prediction=monitor_churn.predict_column
                        )
                ]
)

ref_df = Dataset.from_pandas(
        ref_df,
        data_definition=definition
        )
curr_df = Dataset.from_pandas(
        curr_df,
        data_definition=definition
        )


In [ ]:

report = Report(
    [
        ValueDrift(column=monitor_churn.predict_column, method="psi"),
        # DataDriftPreset(
        #     num_method="psi",
        #     cat_method="psi",
        #     include_tests=True
        # )
    ],
)
my_eval = report.run(current_data=curr_df, reference_data=ref_df)
my_eval

In [ ]:
monitor_churn.check_data_quality(
        reference_df=reference_df, 
        current_df=current_df,
        period=period
        )

In [ ]:
reference_df.as_dataframe()['AVG_TIME_BETWEEN_2_CONTRACTS'].isna().sum()

In [ ]:
monitor_churn.detect_drift(
        reference_df=ref_df, 
        current_df=curr_df,
        period=period
        )

# 2

In [ ]:
monitor_churn_1 = GenericModelMonitor('D:/WORK_F88/Tracy/Projects/001_evidently_monitor_observation/config/monitoring_config.yaml')


In [ ]:
reference_df_pred, current_df_pred = monitor_churn_1.load_data_perform(
        reference_path=reference_path_predict, 
        current_path=current_path_predict
        )
reference_df_pred_pd = pd.read_parquet(reference_path_predict)
current_df_pred_pd = pd.read_parquet(current_path_predict)

In [ ]:
report = Report([
    KSMetric(true_column=monitor_churn_1.target_column, 
             pred_column=monitor_churn_1.predict_column)
])

snapshot = report.run(current_df_pred, reference_df_pred)  # reference_dataset có thể None
snapshot

In [ ]:

monitor_churn_1.evaluate_performance(
        current_df=current_df_pred, 
        reference_df=reference_df_pred, 
        period=period
        )

# Final

In [ ]:
import json
import pandas as pd

reference_df, current_df = monitor_churn.load_data(
        reference_path=reference_path_feature, 
        current_path=current_path_feature
        )

# Tab 1 - Data Quality
result_json_1, ev_1 = monitor_churn.check_data_quality(
    reference_df=reference_df, 
    current_df=current_df,
    period=period
)
# Tab 2 - Drift
drift_json_2, ev_2 = monitor_churn.detect_drift(
        reference_df=reference_df, 
        current_df=current_df,
        period=period
        )
reference_df_pred, current_df_pred = monitor_churn.load_data_perform(
        reference_path=reference_path_predict, 
        current_path=current_path_predict
        )
# Tab 3 - Performance
perf_json_3, ev_3 = monitor_churn.evaluate_performance(
    current_df=current_df_pred, 
    reference_df=reference_df_pred, 
    period=period
)

# ---- Combine 3 ev reports to interactive html tabs ----
def get_html_from_evidently(ev_report):
    """Return HTML content as string from Evidently report object."""
    import tempfile
    tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".html")
    ev_report.save_html(tmp.name)
    with open(tmp.name, "r", encoding="utf-8") as f:
        html = f.read()
    return html

ev1_html = get_html_from_evidently(ev_1)
ev2_html = get_html_from_evidently(ev_2)
ev3_html = get_html_from_evidently(ev_3)

tabbed_html = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8"/>
    <title>Model Monitoring Report</title>
    <style>
        .tab {{
            overflow: hidden;
            border-bottom: 1px solid #ccc;
            background-color: #f1f1f1;
        }}

        .tab button {{
            background-color: inherit;
            border: none;
            outline: none;
            cursor: pointer;
            padding: 10px 20px;
            transition: 0.3s;
            font-size: 16px;
        }}

        .tab button.active {{
            background-color: #e1e1e1;
            font-weight: bold;
        }}

        .tabcontent {{
            display: none;
        }}

        .tabcontent.active {{
            display: block;
        }}

        /* Hide scrollbar on tab content for overlay look */
        .tabcontent {{
            overflow-x: auto;
        }}

    </style>
</head>
<body>
    <h3>Model Monitoring Summary</h3>
    <div class="tab">
        <button class="tablinks active" onclick="openTab(event, 'tab1')">Data Quality</button>
        <button class="tablinks" onclick="openTab(event, 'tab2')">Data Drift</button>
        <button class="tablinks" onclick="openTab(event, 'tab3')">Performance</button>
    </div>

    <div id="tab1" class="tabcontent active">{ev1_html}</div>
    <div id="tab2" class="tabcontent">{ev2_html}</div>
    <div id="tab3" class="tabcontent">{ev3_html}</div>

    <script>
        function openTab(evt, tabName) {{
            // Hide all tabcontent
            var tabcontents = document.getElementsByClassName("tabcontent");
            for (var i = 0; i < tabcontents.length; i++) {{
                tabcontents[i].classList.remove('active');
            }}

            // Remove active state from all buttons
            var tablinks = document.getElementsByClassName("tablinks");
            for (var i = 0; i < tablinks.length; i++) {{
                tablinks[i].classList.remove('active');
            }}

            // Show current tab and set active button
            document.getElementById(tabName).classList.add('active');
            evt.currentTarget.classList.add('active');
        }}
    </script>
</body>
</html>
"""

output_html_path = "model_monitoring_report_tabs.html"
with open(output_html_path, "w", encoding="utf-8") as f:
    f.write(tabbed_html)

print(f"Exported interactive tabbed model monitoring report to {output_html_path}")


# custom metrics

## auc

In [ ]:
import pandas as pd
import numpy as np
from evidently import Report
from evidently import Dataset
from evidently import DataDefinition
from evidently.core.report import Context
from evidently.core.metric_types import SingleValue
from evidently.core.metric_types import SingleValueMetric
from evidently.core.metric_types import SingleValueCalculation
from evidently.core.metric_types import BoundTest
from evidently.tests import Reference, eq
from sklearn.metrics import roc_auc_score

from typing import Optional
from typing import List

class AUCMetric(SingleValueMetric):
    true_column: str
    pred_column: str

    def _default_tests(self) -> List[BoundTest]:
        return [eq(0.5).bind_single(self.get_fingerprint())]

    def _default_tests_with_reference(self) -> List[BoundTest]:
        return [eq(Reference(relative=0.05)).bind_single(self.get_fingerprint())]

# implementation
class AUCMetricImplementation(SingleValueCalculation[AUCMetric]):
    def calculate(self, context: Context, 
        current_data: Dataset, 
        reference_data: Optional[Dataset]
        ) -> SingleValue:
        y_true = current_data.column(self.metric.true_column).data
        y_pred = current_data.column(self.metric.pred_column).data
        auc = roc_auc_score(y_true, y_pred)
        result = self.result(value=auc)
        if reference_data is not None:
            result_ref = self.result(
                value=roc_auc_score(reference_data.column(self.metric.true_column).data,
                                    reference_data.column(self.metric.pred_column).data)
                                    )
            return result, result_ref
        else:
            return result

    def display_name(self) -> str:
        return (
            f"AUC score for {self.metric.true_column} vs {self.metric.pred_column} "
        )

report = Report([
    AUCMetric(true_column=monitor_churn.target_column, 
            pred_column=monitor_churn.predict_column)
])
my_eval = report.run(current_data=current_df_pred, 
                    reference_data=reference_df_pred
                    )
my_eval


In [ ]:
my_eval.json()

## woe

### utils


In [ ]:
SCR_PATH = 'D:/WORK_F88/Tracy/Projects/001_evidently_monitor_observation'
sys.path.append(SCR_PATH)

import importlib
import src.monitoring.utils
importlib.reload(src.monitoring.utils)
from src.monitoring.utils import (load_config,
                                  preprocess_for_woe_iv,
                                  add_label_cols
                                  )


In [1]:
import pandas as pd
df = pd.read_parquet(r'D:\WORK_F88\Tracy\Projects\001_evidently_monitor_observation\data\ref_all.parquet')
df

,CUSTOMER_CODE,DISB,Y_PRED_PROBA,DATE_PARTITION,AVG_TIME_BETWEEN_2_CONTRACTS,AVG_TIME_BETWEEN_3_CONTRACTS,AVG_TIME_BETWEEN_4_CONTRACTS,ASSET_TYPE,CUSTOMER_GENDER,CUSTOMER_AGE,...,NUM_RETURN_CONTRACT_6M,NUM_RETURN_CONTRACT_12M,NUM_TOPUP_CONTRACT_3M,NUM_TOPUP_CONTRACT_6M,NUM_TOPUP_CONTRACT_12M,NUM_CONTRACT_3M,NUM_CONTRACT_6M,NUM_CONTRACT_12M,DISBURSE_DATE_WID,NUM_ACTIVE_CONTRACT
0,1005827,0.0,0.001803,20250625,NaN,NaN,NaN,None,None,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20170119.0,0.0
1,1006510,0.0,0.001967,20250625,31.0,NaN,NaN,None,None,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20170212.0,0.0
2,1007865,0.0,0.001947,20250625,11.0,NaN,NaN,None,None,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20170331.0,0.0
3,1010149,0.0,0.001803,20250625,NaN,NaN,NaN,None,None,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20170529.0,0.0
4,1020618,0.0,0.002118,20250625,NaN,NaN,NaN,None,NAM,31.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20171014.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529406,2599968586,1.0,0.048965,20250625,NaN,NaN,NaN,Đăng ký xe máy,NAM,29.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,20250303.0,0.0
529407,2599976980,0.0,0.077295,20250625,0.0,NaN,NaN,Đăng ký xe máy,NAM,21.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,20250215.0,0.0
529408,2599984166,0.0,0.042992,20250625,NaN,NaN,NaN,Đăng ký xe máy,NAM,31.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,20250119.0,0.0
529409,2599987367,0.0,0.051929,20250625,0.0,NaN,NaN,Đăng ký Ô tô,NỮ,48.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,20250111.0,0.0


In [14]:
import scorecardpy as sc
import numpy as np

# Select features, excluding the target variable(s) if needed
features = df.columns.tolist()

# Assign LABEL as 'good' if DISB == 1, else 'bad' (no lambda)
df['LABEL'] = np.where(df['DISB'] == 1, 'good', 'bad')

# Example: Calculate WOE bins for a single feature (change as needed)
bins = sc.woebin(df, y='LABEL', x=['MAX_DPD_3M'])

[INFO] creating woe binning ...


d:\WORK_F88\Tracy\Projects\001_evidently_monitor_observation\.venv\Lib\site-packages\scorecardpy\condition_fun.py:141: UserWarning: The positive value in "LABEL" was replaced by 1 and negative value by 0.
  warnings.warn("The positive value in \"{}\" was replaced by 1 and negative value by 0.".format(y))
d:\WORK_F88\Tracy\Projects\001_evidently_monitor_observation\.venv\Lib\site-packages\scorecardpy\condition_fun.py:40: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  datetime_cols = dat.apply(pd.to_numeric,errors='ignore').select_dtypes(object).apply(pd.to_datetime,errors='ignore').select_dtypes('datetime64').columns.tolist()
d:\WORK_F88\Tracy\Projects\001_evidently_monitor_observation\.venv\Lib\site-packages\scorecardpy\condition_fun.py:40: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch e

In [15]:
bins

{'MAX_DPD_3M':      variable         bin   count  count_distr  good     bad   badprob  \
 0  MAX_DPD_3M     missing  484369     0.914921  5026  479343  0.989624   
 1  MAX_DPD_3M  [-inf,inf)   45042     0.085079  4645   40397  0.896874   
 
         woe    bin_iv  total_iv   breaks  is_special_values  
 0  0.573595  0.230916  0.964101  missing               True  
 1 -1.821233  0.733186  0.964101      inf              False  }

In [10]:
breaks_list = {}
for feat, bin_df in bins.items():
    if bin_df is not None and len(bin_df) > 0 and 'breaks' in bin_df.columns:
        # Lấy breaks từ bin_df, bỏ qua 'missing' và 'special'
        breaks = bin_df['breaks'].dropna().tolist()
        breaks = [b for b in breaks if b not in ['missing', 'special', 'Missing', 'Special']]
        if breaks:
            breaks_list[feat] = breaks

In [16]:
df['MAX_DPD_3M'].value_counts()

MAX_DPD_3M
-1.0      24603
 0.0       5979
 1.0        810
 2.0        734
-2.0        699
          ...  
 104.0        1
 92.0         1
 132.0        1
 102.0        1
-41.0         1
Name: count, Length: 136, dtype: int64

In [11]:
breaks_list

{'AVG_TIME_BETWEEN_2_CONTRACTS': ['5.0', '45.0', '120.0', 'inf']}

### use

### old_version

In [ ]:
from typing import List, Optional, Dict
import numpy as np
import pandas as pd

from evidently.core.metric_types import (
    SingleValue,
    SingleValueMetric,
    SingleValueCalculation,
    # PlotlyGraph,
    # PlotlyGraphData
)
from evidently import Dataset
from evidently.legacy.renderers.html_widgets import plotly_figure

from evidently.core.report import Context

import scorecardpy as sc
import plotly.graph_objs as go

class IVStrengthRankingMetric(SingleValueMetric):
    target_column: str
    features: List[str]

class IVStrengthRankingMetricImplementation(SingleValueCalculation[IVStrengthRankingMetric]):
    def calculate(
        self,
        context: Context,
        current_data: Dataset,
        reference_data: Optional[Dataset],
    ) -> SingleValue:
        target_col = self.metric.target_column
        features = self.metric.features

        def _get_iv_table(df: pd.DataFrame, target_col: str, features: List[str]) -> pd.DataFrame:
            raw_cols = df.columns.tolist()
            df = df.copy()
            df.columns = df.columns.str.strip()
            target_col_strip = target_col.strip()

            df['LABEL'] = ['good' if x == 1 else 'bad' for x in df[target_col_strip]]
            features = [c for c in features if c != target_col_strip and c in df.columns]
            drop_cols = ['CUSTOMER_CODE', 'DATE_PARTITION']
            df = df.drop(columns=[c for c in drop_cols if c in df.columns], errors="ignore")

            # calculate bins and woe transforms
            bins = sc.woebin(df, y='LABEL', x=features)
            iv_result = sc.iv(df, y='LABEL', x=features)
            if "iv" not in iv_result.columns and "info_value" in iv_result.columns:
                iv_result = iv_result.rename(columns={'info_value': 'iv'})

            # WOE value for display in table
            def _rank_IV(iv):
                if iv <= 0.02:
                    return 'Useless'
                elif iv <= 0.1:
                    return 'Weak'
                elif iv <= 0.3:
                    return 'Medium'
                elif iv <= 0.5:
                    return 'Strong'
                else:
                    return 'Suspicious'

            iv_result['rank'] = iv_result['iv'].apply(_rank_IV)
            iv_result = iv_result.sort_values('iv', ascending=False)
            iv_result = iv_result.rename(columns={'variable': 'column', 'iv': 'IV'})
            iv_result = iv_result[['column', 'IV', 'rank']]
            return iv_result.reset_index(drop=True), bins

        # Calculate IV tables and bins for current and reference datasets
        cur_df = current_data.as_dataframe()
        cur_iv_table, cur_bins = _get_iv_table(cur_df, target_col, features)

        if reference_data is not None:
            ref_df = reference_data.as_dataframe()
            ref_iv_table, ref_bins = _get_iv_table(ref_df, target_col, features)

            comp = pd.merge(ref_iv_table, cur_iv_table, on='column', suffixes=('_ref', '_cur'))
            comp['IV_change'] = comp['IV_cur'] - comp['IV_ref']
            comp['rank_change'] = comp['rank_cur'] + "←" + comp['rank_ref']
            comp = comp[['column', 'IV_ref', 'rank_ref', 'IV_cur', 'rank_cur', 'IV_change', 'rank_change']]
            iv_table = comp
        else:
            iv_table = cur_iv_table

        iv_table_value = iv_table.to_dict(orient="records")

        # For the top variable(s), plot WOE curve (univariate)
        # For each feature (top 1 or several), plot WOE vs bin
        plotly_figs = []
        top_features = iv_table['column'].values[:3] if len(iv_table) >= 3 else iv_table['column'].values
        bins_to_plot = cur_bins  # use current data bins for the plot

        for feat in top_features:
            # get the binning dataframe for variable
            bin_df = bins_to_plot.get(feat)
            if bin_df is None:
                continue
            
            # Prepare x: bin intervals/levels, y: woe, Count, %Bad, etc.
            x_labels = bin_df['bin'].astype(str)
            y_woe = bin_df['woe'].astype(float)
            y_goodrate = bin_df['good'].astype(float)
            y_badrate = bin_df['bad'].astype(float)
            y_count = bin_df['count']

            fig = go.Figure()
            # WOE
            fig.add_trace(
                go.Bar(
                    x=x_labels,
                    y=y_woe,
                    name="WOE",
                    marker_color="darkorange",
                    yaxis="y1"
                )
            )
            # %Bad rate
            fig.add_trace(
                go.Scatter(
                    x=x_labels,
                    y=y_badrate,
                    name="Bad Rate",
                    mode="lines+markers",
                    yaxis="y2",
                    line=dict(color="crimson", dash="solid"),
                )
            )
            # %Good rate
            fig.add_trace(
                go.Scatter(
                    x=x_labels,
                    y=y_goodrate,
                    name="Good Rate",
                    mode="lines+markers",
                    yaxis="y2",
                    line=dict(color="royalblue", dash="dot")
                )
            )
            # Compose layout
            fig.update_layout(
                title=f"WOE & Bad Rate by Group: {feat}",
                xaxis_title="Bin",
                yaxis=dict(
                    title="WOE",
                    side="left",
                    showgrid=False
                ),
                yaxis2=dict(
                    title="Good/Bad Rate",
                    overlaying="y",
                    side="right",
                    showgrid=False
                ),
                barmode="group",
                legend_title="Stat",
                template="plotly_white",
                margin=dict(l=40, r=40, t=60, b=40),
                height=400
            )
            result = self.result(value=iv_table_value)
            result.widget = [plotly_figure(title=self.display_name(), figure=fig)]


        return self.result(
            value=iv_table_value,
            figure=plotly_figs if plotly_figs else None
        )

    def display_name(self) -> str:
        return "Biến xếp hạng theo IV (IV Strength & Rank by Reference, Compared with Current)\nBiểu đồ WOE & Bad Rate của các biến lớn"

# Sử dụng trong Evidently (có hình):
# report = Report([IVStrengthRankingMetric(target_column='BAD', features=list_of_features)])
# result = report.run(current_data=..., reference_data=...)
# result.json() sẽ chứa bảng IV/strength/ref/cur và figure WOE các biến top


In [ ]:
# 1. Reload module
import importlib
import src.monitoring.pipeline
importlib.reload(src.monitoring.pipeline)
from src.monitoring.pipeline import GenericModelMonitor

# 2. Tạo lại monitor instance
monitor_churn_1 = GenericModelMonitor('D:/WORK_F88/Tracy/Projects/001_evidently_monitor_observation/config/monitoring_config.yaml')

# 3. Load lại data (TẠO DATASET MỚI)
all_ref_df, all_cur_df = monitor_churn_1.load_data(
    reference_path=r'D:\WORK_F88\Tracy\Projects\001_evidently_monitor_observation\notebooks\ref_all.parquet', 
    current_path=r'D:\WORK_F88\Tracy\Projects\001_evidently_monitor_observation\notebooks\cur_all.parquet'
)


report = Report([
    IVStrengthRankingMetric(
        target_column=monitor_churn.target_column,
        features=monitor_churn.numerical_columns,
    )
])
# 5. Chạy report
report.run(current_data=all_cur_df, reference_data=all_ref_df)

In [ ]:
import scorecardpy as sc
import pandas as pd
from typing import List

def _cal_woe_n_iv(df: pd.DataFrame, target_col: str, features: List[str], iv_threshold: float):
    df = df.drop(columns=['CUSTOMER_CODE', 'DATE_PARTITION'], errors='ignore')
    df['LABEL'] = ['good' if x == 1 else 'bad' for x in df[target_col]]
    df.columns = df.columns.str.strip()
    if target_col not in df.columns:
        raise KeyError(f"Target column '{target_col}' not found. Available columns sample: {list(df.columns)[:30]}")
    features = [c for c in features if c != target_col]

    print(f"Features: {features}")

    bins = sc.woebin(df, y='LABEL', x=features)
    return bins


df = pd.read_parquet(r'D:\WORK_F88\Tracy\Projects\001_evidently_monitor_observation\notebooks\ref_all.parquet')

# Giả sử features = monitor_churn.numerical_columns (phải tồn tại ở đây)  
# Nếu chưa có biến "features" đúng, cần tự định nghĩa hoặc lấy từ class monitor_churn

bins= _cal_woe_n_iv(df, 'DISB', features, 0.02)


In [ ]:
bins['N_DPD_30D_3M']

In [ ]:
from typing import List, Optional
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import scorecardpy as sc

from evidently.core.metric_types import SingleValue, SingleValueMetric, SingleValueCalculation
from evidently.core.report import Context
from evidently import Dataset
from evidently.legacy.renderers.html_widgets import plotly_figure

# Thêm logic LABEL theo rule như cell ở đầu
def add_label_col(df: pd.DataFrame, target_col: str):
    # Gọi trực tiếp logic phân loại LABEL good/bad như ở cell đầu file
    df = df.copy()
    df.columns = df.columns.str.strip()
    if target_col not in df.columns:
        raise KeyError(f"Target column '{target_col}' not found. Available columns: {list(df.columns)[:30]}")
    # Lấy lại logic mapping
    df['LABEL'] = ['good' if x == 1 else 'bad' for x in df[target_col]]
    return df

class BasicWOEMetric(SingleValueMetric):
    target_column: str           # y: 0/1 (bad=1)
    features: List[str]          # list feature columns
    top_n: int = 50              # show top N features (by order), for readability

class BasicWOEMetricImplementation(SingleValueCalculation[BasicWOEMetric]):
    def _preprocess_df(self, df: pd.DataFrame, target: str, features: List[str]) -> tuple:
        # Strip all column names and drop unwanted columns
        df = df.copy()
        df.columns = df.columns.str.strip()
        df = df.drop(columns=["CUSTOMER_CODE", "DATE_PARTITION"], errors="ignore")

        # Ensure target exists
        if target not in df.columns:
            raise KeyError(f"Target column '{target}' not found after column cleanup. Available columns: {list(df.columns)[:30]}")

        # Only keep valid features and target
        selected_features = [c for c in features if c in df.columns and c != target]
        # Thêm LABEL cho đúng expectation scorecardpy
        df = add_label_col(df, target)
        df = df[selected_features + ["LABEL"]]

        return df, selected_features

    def _calc_woe(self, df, features, target):
        # Process DataFrame before calculation
        df_proc, used_features = self._preprocess_df(df, target, features)

        # scorecardpy hates NA -> drop rows with NA in any used features
        work = df_proc.dropna(subset=used_features)
        # Run woebin (dùng LABEL)
        bins = sc.woebin(work, y='LABEL', x=used_features)
        return bins, used_features

    def calculate(
        self,
        context: Context,
        current_data: Dataset,
        reference_data: Optional[Dataset],
    ) -> SingleValue:
        features = [c.strip() for c in self.metric.features]
        target_col = self.metric.target_column

        # ---- current_data ----
        df_cur = current_data.as_dataframe()
        bins_cur, use_feats_cur = self._calc_woe(df_cur, features, target_col)
        show_feats_cur = use_feats_cur[:self.metric.top_n]

        # ---- reference_data ---- 
        bins_ref = None
        show_feats_ref = None
        if reference_data is not None:
            df_ref = reference_data.as_dataframe()
            bins_ref, use_feats_ref = self._calc_woe(df_ref, features, target_col)
            show_feats_ref = use_feats_ref[:self.metric.top_n]

        widgets = []
        # Display in two columns: left=current, right=reference
        from plotly.subplots import make_subplots
        top_feats = show_feats_cur[:10]  # Lấy tối đa 10 features để hiển thị

        for f in top_feats:
            bin_cur = bins_cur.get(f)
            bin_ref = bins_ref.get(f) if (bins_ref is not None and f in bins_ref) else None

            if (bin_cur is None or len(bin_cur) == 0) and (bin_ref is None or len(bin_ref) == 0):
                continue

            # find bad rate column robustly cho cả current/ref
            def find_bad_col(df_bin):
                if df_bin is None:
                    return None
                for c in ["badprob", "bad_rate", "badrate"]:
                    if c in df_bin.columns:
                        return c
                return None

            bad_col_cur = find_bad_col(bin_cur)
            bad_col_ref = find_bad_col(bin_ref)

            x_cur = bin_cur["bin"].astype(str) if (bin_cur is not None and "bin" in bin_cur.columns) else None
            x_ref = bin_ref["bin"].astype(str) if (bin_ref is not None and "bin" in bin_ref.columns) else None

            # Sanity check: dùng bin từ current nếu bin_ref bị thiếu hoặc khác
            x_vals = x_cur if x_cur is not None else x_ref
            subplot_titles = ["Current", "Reference"]

            fig = make_subplots(
                rows=1, cols=2, shared_yaxes=False,
                subplot_titles=subplot_titles
            )

            # Vẽ current bên trái
            if bin_cur is not None and len(bin_cur):
                woe_cur = pd.to_numeric(bin_cur["woe"], errors="coerce") if "woe" in bin_cur.columns else None
                badrate_cur = pd.to_numeric(bin_cur[bad_col_cur], errors="coerce") if bad_col_cur else None

                fig.add_trace(
                    go.Bar(x=x_cur, y=woe_cur, name="WOE", yaxis="y1", marker_color="#1f77b4"),
                    row=1, col=1,
                )
                fig.add_trace(
                    go.Scatter(x=x_cur, y=badrate_cur, name="Bad rate", mode="lines+markers",
                               marker_color="#ff7f0e", yaxis="y2"),
                    row=1, col=1,
                )
            # Vẽ reference bên phải
            if bin_ref is not None and len(bin_ref):
                woe_ref = pd.to_numeric(bin_ref["woe"], errors="coerce") if "woe" in bin_ref.columns else None
                badrate_ref = pd.to_numeric(bin_ref[bad_col_ref], errors="coerce") if bad_col_ref else None

                fig.add_trace(
                    go.Bar(x=x_ref, y=woe_ref, name="WOE", yaxis="y1", marker_color="#1f77b4", showlegend=False),
                    row=1, col=2,
                )
                fig.add_trace(
                    go.Scatter(x=x_ref, y=badrate_ref, name="Bad rate", mode="lines+markers",
                               marker_color="#ff7f0e", yaxis="y2", showlegend=False),
                    row=1, col=2,
                )

            # Layout cho 2 cột
            fig.update_layout(
                title=f"WOE & Bad Rate by Bin — {f}",
                height=420,
                margin=dict(l=40, r=40, t=60, b=60),
            )
            fig.update_xaxes(title_text="Bin", row=1, col=1)
            fig.update_xaxes(title_text="Bin", row=1, col=2)
            fig.update_yaxes(title_text="WOE", row=1, col=1, side="left")
            fig.update_yaxes(title_text="WOE", row=1, col=2, side="left")
            # Nếu muốn Bad rate sang trục phải thì cần define thêm yaxis2... 
            # (phức tạp, có thể skip cụ thể về multi-axis nếu cần)

            widgets.append(plotly_figure(title=f"WOE: {f} (Current / Reference)", figure=fig))

        # --- result ---
        result_ref = None
        if reference_data is not None and bins_ref is not None:
            result_ref = self.result(value=1)

        result = self.result(value=1)
        result.widget = widgets

        if reference_data is not None and result_ref is not None:
            return result, result_ref
        else:
            return result

    def display_name(self) -> str:
        return "Basic WOE per feature"


In [ ]:
import importlib
import src.monitoring.pipeline
importlib.reload(src.monitoring.pipeline)
from src.monitoring.pipeline import GenericModelMonitor

monitor_churn_1 = GenericModelMonitor(
    'D:/WORK_F88/Tracy/Projects/001_evidently_monitor_observation/config/monitoring_config.yaml'
)

all_ref_df, all_cur_df = monitor_churn_1.load_data(
    reference_path=r'D:\WORK_F88\Tracy\Projects\001_evidently_monitor_observation\notebooks\ref_all.parquet',
    current_path=r'D:\WORK_F88\Tracy\Projects\001_evidently_monitor_observation\notebooks\cur_all.parquet'
)

# (Optional) sanity checks: cột target + vài feature có tồn tại không
def get_columns(obj):
    if hasattr(obj, "columns"):
        return obj.columns
    if hasattr(obj, "get_table_columns"):
        try:
            return obj.get_table_columns()
        except Exception:
            pass
    if hasattr(obj, "_data") and hasattr(obj._data, "columns"):
        return obj._data.columns
    return []

ref_cols = get_columns(all_ref_df)
cur_cols = get_columns(all_cur_df)

print("Target:", monitor_churn_1.target_column)
print("Example feature count:", len(monitor_churn_1.numerical_columns))
print("Target in current?", monitor_churn_1.target_column in cur_cols)
print("Target in reference?", monitor_churn_1.target_column in ref_cols)

from evidently import Report

report = Report([
    BasicWOEMetric(
        target_column=monitor_churn_1.target_column,     # y (thường bad=1)
        features=monitor_churn_1.numerical_columns,      # list feature numeric
    )
])

snapshot = report.run(current_data=all_cur_df, reference_data=all_ref_df)


In [ ]:
snapshot

In [ ]:
class IVSummaryMetric(SingleValueMetric):
    target_column: str
    numeric_features: List[str]
    categorical_features: List[str]
    top_n: int = 30


class IVSummaryMetricImplementation(SingleValueCalculation[IVSummaryMetric]):
    def calculate(self, context: Context, current_data: Dataset, reference_data: Optional[Dataset]) -> SingleValue:
        tgt = self.metric.target_column.strip()
        num = self.metric.numeric_features
        cat = self.metric.categorical_features

        cur = current_data.as_dataframe().drop(columns=["CUSTOMER_CODE", "DATE_PARTITION"], errors="ignore")

        # Use reference medians if possible (stable fill)
        ref_meds = None
        if reference_data is not None:
            ref = reference_data.as_dataframe().drop(columns=["CUSTOMER_CODE", "DATE_PARTITION"], errors="ignore")
            ref_prep, ref_meds = prepare_ab(ref, tgt, num, cat)
        cur_prep, _ = prepare_ab(cur, tgt, num, cat, ref_medians=ref_meds)

        # Features include numeric missing flags
        feats = [c for c in (num + [f"{c}_isna" for c in num] + cat) if c in cur_prep.columns and c != tgt]

        iv = sc.iv(cur_prep, y=tgt, x=feats)
        if "iv" not in iv.columns and "info_value" in iv.columns:
            iv = iv.rename(columns={"info_value": "iv"})
        iv = iv.rename(columns={"variable": "feature"})[["feature", "iv"]].sort_values("iv", ascending=False).reset_index(drop=True)

        mean_iv = float(iv["iv"].mean()) if len(iv) else np.nan
        pct_iv_low = float((iv["iv"] < 0.02).mean()) if len(iv) else np.nan

        result = self.result(value=mean_iv)

        top = iv.head(self.metric.top_n)
        fig = go.Figure([go.Bar(x=top["feature"], y=top["iv"], name="IV")])
        fig.update_layout(
            title=f"IV Summary | mean_iv={mean_iv:.4f} | pct_iv_low={pct_iv_low:.2%}",
            xaxis_title="Feature",
            yaxis_title="IV",
            height=420,
            margin=dict(l=40, r=20, t=60, b=130),
        )

        result.widget = [plotly_figure(title="IV Summary", figure=fig)]
        return result

    def display_name(self) -> str:
        return "IV Summary"

In [ ]:
import importlib
import src.monitoring.pipeline
importlib.reload(src.monitoring.pipeline)
from src.monitoring.pipeline import GenericModelMonitor

monitor_churn_1 = GenericModelMonitor(
    'D:/WORK_F88/Tracy/Projects/001_evidently_monitor_observation/config/monitoring_config.yaml'
)

all_ref_df, all_cur_df = monitor_churn_1.load_data(
    reference_path=r'D:\WORK_F88\Tracy\Projects\001_evidently_monitor_observation\notebooks\ref_all.parquet',
    current_path=r'D:\WORK_F88\Tracy\Projects\001_evidently_monitor_observation\notebooks\cur_all.parquet'
)

# (Optional) sanity checks: cột target + vài feature có tồn tại không
def get_columns(obj):
    # Try .columns if obj is a DataFrame
    if hasattr(obj, "columns"):
        return obj.columns
    # If obj is a PandasDataset (Great Expectations), try .get_table_columns()
    if hasattr(obj, "get_table_columns"):
        try:
            return obj.get_table_columns()
        except Exception:
            pass
    # Try to access _data if present (sometimes in wrappers)
    if hasattr(obj, "_data") and hasattr(obj._data, "columns"):
        return obj._data.columns
    return []

ref_cols = get_columns(all_ref_df)
cur_cols = get_columns(all_cur_df)

print("Target:", monitor_churn_1.target_column)
print("Example feature count:", len(monitor_churn_1.numerical_columns))
print("Target in current?", monitor_churn_1.target_column in cur_cols)
print("Target in reference?", monitor_churn_1.target_column in ref_cols)

from evidently import Report

report = Report([
    IVSummaryMetric(
        target_column=monitor_churn_1.target_column,     # y (thường bad=1)
        numeric_features=monitor_churn_1.numerical_columns,      # list feature numeric
        categorical_features=monitor_churn_1.categorical_columns
    )
])

snapshot = report.run(current_data=all_cur_df, reference_data=all_ref_df)


In [ ]:
snapshot